In [ ]:
import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import keras

!pip install git+https://github.com/google/qkeras

  Cloning https://github.com/google/qkeras to /tmp/pip-req-build-al85ld7b
  Running command git clone -q https://github.com/google/qkeras /tmp/pip-req-build-al85ld7b
     |████████████████████████████████| 174kB 6.0MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 22.2MB 1.4MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
  Created wheel for QKeras: filename=QKeras-0.8.0-cp36-none-any.whl size=148269 sha256=c4fbdc331fd4c2fd9f4c396d738ad0bb0631f056c5805f653d8c841867dbb30d
  Stored in directory: /tmp/pip-ephem-wheel-cache-q4aromrg/wheels/b4/74/1d/9456d62789716894a5edd7e342b4beaef69241ac584706c68d
  Created wheel for pyparser: filename=pyparser-1.0-cp36-none-any.whl size=4944 sha256=2cfe95f651e8d7a550e542d38574ae0d29dc8049b1ea731539650cca829c12c4
  Stored in directory: /root/.cache/pip/wheels/8a/1c/4f/9f66cd69719aa41c2684efae758d95db7803e9fe1f65146db1
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl 

In [ ]:
def get_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,)).astype("float32")
    x_test = x_test.reshape(x_test.shape + (1,)).astype("float32")

    x_train /= 256.0
    x_test /= 256.0

    x_mean = np.mean(x_train, axis=0)

    x_train -= x_mean
    x_test -= x_mean

    nb_classes = np.max(y_train)+1
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)

    quantizer = quantized_bits(9, 1)
    x_train = quantizer(x_train).numpy()
    x_test = quantizer(x_test).numpy()

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = get_data()

In [ ]:
### Here I'll try to use the method outlined in this link
# https://stackoverflow.com/questions/54204393/piecewise-activation-function-in-tensorflow-and-broadcasting-math-operation

def swish(x):
  global intervals, coeffArray
  #return x
  coeff = np.array(coeffArray)
  conditionArray = sum([tf.multiply(tf.cast(tf.math.logical_and(tf.math.less(x, 0.03125*(n+1)), tf.math.greater_equal(x, 0.03125*n)), tf.float32), coeff[n][0]*x + coeff[n][1]*K.ones_like(x)) for n in range(256)])

  #return quantized_bits(17, 1)(conditionArray)
  return conditionArray


In [ ]:
from qkeras import *
import qkeras
def CreateModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)
    x = Flatten(name="flatten")(x)

    #x = QDense(256,
    #    kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
    #    bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
    #    name="dense")(x)
    x = Dense(256)(x)

    #x = MyActivation()(x)
    x = Activation(swish)(x)

    #x = QDense(128,
    #    kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
    #    bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
    #    name="dense2")(x)
    x = Dense(128)(x)

    #x = MyActivation()(x)
    x = Activation(swish)(x)

    #x = QDense(128,
    #    kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
    #    bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
    #    name="dense3")(x)
    x = Dense(128)(x)

    #x = MyActivation()(x)
    x = Activation(swish)(x)

    #x = QDense(nb_classes,
    #    kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
    #    bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
    #    name="dense4")(x)
    x = Dense(nb_classes)(x)
    
    x = Activation("softmax", name="softmax")(x)

    model = Model(inputs=x_in, outputs=x)    
    return model

In [ ]:
from tensorflow.keras.optimizers import Adam
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)

model = CreateModel(x_train.shape[1:], y_train.shape[-1], 1)
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(0.0005),
    #optimizer='sgd',
    metrics=["accuracy"],)
history = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test[:5000], y_test[:5000]), verbose=False, callbacks=[callback])



In [ ]:
from qkeras.utils import *
model_save_quantized_weights(model)
print("Done")

In [ ]:
model.evaluate(x_test[5000:], y_test[5000:])

# PWLAF

e --> Error
r --> Input Range


In [ ]:
from scipy.optimize import fsolve

In [ ]:
#Given
e = 0.04
r = 8

#Number of input bits
Ninp = np.ceil(np.log(r)/np.log(2)) + np.ceil(-1*np.log(e)/np.log(2))

#Number of Output/Fractional bits
Nout = np.ceil(-1*np.log(e)/np.log(2))

#Boundaries
def f(x):
  global e
  return (x**3)/3 - (2*(x**5)/15 + e)

xpa = fsolve(f, [1])[0]
xpaq = r*np.ceil(xpa* (2**(Ninp))/r)/(2**Ninp)

temp = 1-(e + 2**(-1*Nout))
xs = np.arctanh(temp)
xsq = r*np.ceil(xs* (2**(Ninp))/r)/(2**Ninp)

#Permissible approximation error
ea = e - 2**(-1*(Nout+1))

In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-1*x))

def tanh(x):
  return np.tanh(x)

In [ ]:
import scipy.integrate as integrate
import scipy.special as special
result = integrate.quad(lambda x: tanh(x), 0, 4.5)
result[0]

3.806976221629778

We divide the given range into 256 equal segments. In each segment, we wish to make the area equal to that under the actual function.

In each sub-interval we choose from a set of further 100 points, to determine thw optimal values of the constants c1 and c2

In [ ]:
given_range = [-4, 4]

intervals = np.linspace(given_range[0], given_range[1], 257)
from qkeras import *
quantizer = quantized_bits(9, 3)
intervals = sorted(list(set(quantizer(intervals).numpy())))
#intervals.append(8)

In [ ]:
len(intervals)

257

In [ ]:
intervals[::-1][:10]

[4.0,
 3.968628,
 3.9372559,
 3.9058838,
 3.8745117,
 3.8431396,
 3.8117676,
 3.7803955,
 3.7490234,
 3.7176514]

In [ ]:
from scipy import linalg
import math
def findOptimalPoint(a, b, func):
  ans = []
  error = []
  #print (error)
  compare1 = np.array([sigmoid(x) for x in np.linspace(a,b, 10)])
  for c in np.linspace(a, b, 10):
    
    if c == a or c==b:
      pass
    TrueArea1 = integrate.quad(lambda x: func(x), a, c)[0]
    TrueArea2 = integrate.quad(lambda x: func(x), c, b)[0]
    
    #print (TrueArea1, TrueArea2)
    
      #solution = linalg.solve(np.array([[0.5*c**2 - 0.5*a**2, c-a], [0.5*b**2 - 0.5*c**2, b-c]]), np.array([TrueArea1, TrueArea2]))
      #c1, c2 = solution[0], solution[1]

    den1 = 0.5*(a-b)*(b-c)*(c-a)
    num1 = (b-c)*TrueArea1 - (c-a)*TrueArea2
    c1 = num1/den1
    #print (den1, c1)

    den2 = (a-b)*(b-c)*(c-a)
    num2 = (c**2 - a**2)*TrueArea2 - (b**2 - c**2)*TrueArea1
    c2 = num2/den2
    #print (den2, c2)

    if (den1 == 0 or den2 == 0):
      pass

    compare2 = np.array([c1*x + c2 for x in np.linspace(a, b, 10)])
    #print (compare1)

    e = np.max(np.abs(compare1 - compare2))
    #print (e)
    if math.isnan(e):
      #print ("Nan detected")
      pass
    if len(error) > 0:
        if error[0] > e:
          ans = [c, c1, c2]
          error[0] = e
    else:
      if math.isnan(e) != True:
        ans = [c, c1, c2]
        error = [e]

    #rint (e, error, math.isnan(e))
  return ans[0], ans[1], ans[2]    

In [ ]:
coeffArray = []
for i in range(256):
  #print (i)
  c, c1, c2 = findOptimalPoint(intervals[i], intervals[i+1], tanh)
  coeffArray.append([c1, c2])

  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
coeffArray[50:55]

[[0.074911400252175, 0.26294835338288697],
 [0.07689065543753462, 0.2677097120412446],
 [0.07891035288601293, 0.2725052456518795],
 [0.08097062090447397, 0.2773327273068449],
 [0.08307154112310883, 0.28218981016199146]]

In [ ]:
np.searchsorted(intervals, [[1, 2], [4, 5]])

array([[ 32,  64],
       [128, 160]])